In [1]:
#clone repository

!git clone -b AI_2 https://github.com/SWhack-PhotoWithAI/PWA_APP.git
%cd PWA_APP/

!pip install flask-ngrok
!pip install -r NIMA/requirement.txt

Cloning into 'PWA_APP'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 846 (delta 63), reused 191 (delta 37), pack-reused 611
Receiving objects: 100% (846/846), 62.14 MiB | 39.67 MiB/s, done.
Resolving deltas: 100% (164/164), done.
/content/PWA_APP
  Created wheel for random-name: filename=random_name-0.1.1-cp36-none-any.whl size=5474 sha256=683688746fa7f61a9d6161b7a93d4c78c6f1b0ce2929faaf1e025290fc6f40d9
  Stored in directory: /root/.cache/pip/wheels/17/fc/c5/abe2d2ee67eef0c0606d96ea02157c6c8079e32418da62b942
Successfully built random-name


In [3]:
#run Flask WebServer

#Vision Module
from NIMA.evaluate_NIMA import evaluate
from Human.face_based import main
from Human.rt_eval import main as rt_main
#Flask 

from flask_ngrok import run_with_ngrok
# from flask import Flask
from flask import Flask, render_template, request, jsonify
from werkzeug.utils import secure_filename

import json
import os
import shutil
import requests

if 'images' not in os.listdir():
  os.mkdir('images')

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route('/predict_background', methods = ['POST'])
def predict_background(): #image 파일은 post형식으로 넘어오기 때문에
    #제일 score값이 좋은 image name을 return 하면 될듯.
    img_dir = 'images/'
    files = request.files.getlist('image') #이미지 여러개 받아오도록 해야함
    order_dict = {}
    index = 0
    for file in files:
        img_path = img_dir + secure_filename(file.filename)
        order_dict[img_path] = index
        index += 1 
        file.save(img_path)
    
    result = evaluate()
    result_index = order_dict[result['name']]

    _dict = {}
    _dict['index'] = result_index
    return _dict

@app.route('/predict_person', methods = ['POST'])
def predict_person():
    img_dir = 'images/'
    files = request.files.getlist('image') #이미지 여러개 받아오도록 해야함
    
    order_dict = {}
    index = 0
    for file in files:
        img_path = img_dir + secure_filename(file.filename)
        order_dict[img_path] = index
        index += 1
        file.save(img_dir + secure_filename(file.filename))

    result = main()
    best_img = ''
    max_score = -1
    for img_name, score in result.items():
        if score > max_score:
            best_img = img_name

    result_index = order_dict[best_img]

    _dict = {}
    _dict['index'] = result_index
    return _dict

@app.route('/predict_person_rt', methods = ['POST'])
def predict_person_rt():
    img_dir = 'images/'
    files = request.files.getlist('image') #이미지 여러개 받아오도록 해야함

    for file in files:
        img_path = img_dir + secure_filename(file.filename)
        file.save(img_dir + secure_filename(file.filename))

    result = rt_main()

    _dict = {}
    _dict['sen'] = result
    return _dict

# @app.route('/cartoonization', methods = ['POST'])
# def cartoonization():
#     img_paths = []
#     img_dir = 'images/'
#     files = request.files.getlist('source') #이미지 여러개 받아오도록 해야함
#     for file in files:
#         img_path = img_dir + secure_filename(file.filename)
#         print(img_path)
#         img_paths.append(img_path)
#         file.save(img_path)
    
#     for img_path in img_paths:  
#         path = img_dir + 'test.jpg'
#         url = 'https://master-white-box-cartoonization-psi1104.endpoint.ainize.ai/predict'
#         _file = {'source': open(img_path, 'rb'), 'file_type' : 'image'}
#         _response = requests.post(url, files=_file)
#         print(_response)
#         if _response.status_code == 200:
#             with open(path, 'wb') as f:
#                 _response.raw.decode_content = True
#                 shutil.copyfileobj(_response.raw, f)
#     _dict = {}
#     _dict['index'] = 'test'
#     return _dict

@app.route("/")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://426c8c6ac134.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/Feb/2021 22:42:53] "POST /predict_persom_rt HTTP/1.1" 404 -


Loading images from directory :  images/


127.0.0.1 - - [05/Feb/2021 22:43:17] "POST /predict_person_rt HTTP/1.1" 200 -
